# Modelos alternativos - CNNs

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !unzip '/content/drive/MyDrive/datasets/bargal/dataset.processed.GRLogDiff.zip' -d './data/processed'

In [3]:
# # Uncomment this if running in Google Colab. It will install the bargal package from GitHub.
# !pip install git+https://github.com/ludanortmun/itesm-mna-barred-galaxies.git

## Importando librerías

In [4]:
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import torch
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image

from bargal.dataset.load import load_dataset

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


## Preparando el conjunto de datos

El primer paso consiste en cargar nuestro conjunto de datos y dividirlo en conjunto de entrenamiento, validación y prueba.

In [6]:
dataset_path = '../data/dataset.csv'

df = load_dataset(dataset_path)

train_df, val_test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['Bars'])
valid_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42, stratify=val_test_df['Bars'])

print(f'Train size: {len(train_df)}')
print(f'Validation size: {len(valid_df)}')
print(f'Test size: {len(test_df)}')

Train size: 7088
Validation size: 1519
Test size: 1519


Ahora creamos nuestra clase para representar el conjunto de datos, heredando de Dataset de PyTorch. El dataset a utilizar para nuestra red neuronal consiste en imágenes de galaxias y su respectiva etiqueta, que indica si la galaxia tiene o no una barra.

La etiqueta es un tensor de tamaño 1, donde 1 indica que la galaxia tiene una barra y 0 indica que no. Sin embargo, el conjunto de datos original no tiene etiquetas binarias, sino múltiples categorías indicando el tipo de barra que tiene la galaxia. Por lo tanto, convertimos cualquier etiqueta que represente la presencia de una barra (independientemente de sus características) a 1 y cualquier etiqueta que represente la ausencia de una barra a 0. También debemos filtrar elementos con la etiqueta -0.5, la cual simboliza que la categoria de la galaxia es desconocida. Estas transformaciones fueron exploradas previamente en el entregable [Avance1.Equipo22.ipynb](https://github.com/ludanortmun/itesm-mna-barred-galaxies/tree/main/notebooks/Avance1.Equipo22.ipynb), donde se creó la columna `has_bar` derivada de `Bars` y se filtraron los elementos sin clasificación.

En cuanto a la carga de imágenes, este conjunto de datos consiste en las imágenes de galaxias previamente procesadas en formato PNG. El preprocesamiento consiste, principalmente, en la sustracción de las bandas G y R para enfatizar las estructuras de barras. Las imágenes resultantes tienen dimensiones de 400x400 píxeles y están en escala de grises. Estas imágenes son cargadas utilizando la librería PIL y convertidas a tensores.

El script de preprocesamiento puede ser consultado en este enlace: [bargal/commands/preprocess.py](https://github.com/ludanortmun/itesm-mna-barred-galaxies/blob/297f69b278ea6bc5099ef23a0d539602995bc55e/bargal/commands/preprocess.py)

El conjunto de imágenes pre procesadas puede descargarse con el siguiente enlace: [dataset.processed.GRLogDiff](https://tecmx-my.sharepoint.com/:u:/g/personal/a01795197_tec_mx/EexaLnqaLLdCt1JNxLib8VYBeOHJo95vuOr-Pfxv-55Iww?e=0gfeuq)


In [7]:
class GalaxiesDataset(Dataset):
    def __init__(self, galaxies_df: pd.DataFrame, img_dir: str):
        self.filtered_df = galaxies_df[galaxies_df['Bars'] >= 0].reset_index(drop=True)
        self.img_dir = img_dir

    def __len__(self):
        return len(self.filtered_df)

    def __getitem__(self, idx):
        row = self.filtered_df.iloc[idx]
        label = 1.0 if row['Bars'] != 0 else 0.0
        img_path = f"{self.img_dir}/{row['name']}.png"

        # Load image and convert to tensor
        with Image.open(img_path) as img:
            image = transforms.ToTensor()(img)

        return image.to(device), torch.tensor([label], dtype=torch.float32).to(device)

In [16]:
n = 32
processed_images_path = r'D:\datasets\bargal\processed'

train_data = GalaxiesDataset(train_df, processed_images_path)
train_loader = DataLoader(train_data, batch_size=n, shuffle=True)
train_N = len(train_loader.dataset)

valid_data = GalaxiesDataset(valid_df, processed_images_path)
valid_loader = DataLoader(valid_data, batch_size=n)
valid_N = len(valid_loader.dataset)

test_data = GalaxiesDataset(test_df, processed_images_path)
test_loader = DataLoader(test_data, batch_size=n)
test_N = len(test_loader.dataset)

## Definiendo el modelo

### Modelo pre-entrenado base

Para este modelo, se utilizará la arquitectura MobileNet V3 en su variante Large pre-entrenada con el conjunto de datos de ImageNet.

In [9]:
from torchvision.models.mobilenet import mobilenet_v3_large
from torchvision.models import MobileNet_V3_Large_Weights

weights = MobileNet_V3_Large_Weights.DEFAULT
mobilenet_model = mobilenet_v3_large(weights=weights)

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to C:\Users\danie/.cache\torch\hub\checkpoints\mobilenet_v3_large-5c1a4163.pth


100.0%


In [10]:
mobilenet_model.to(device)
mobilenet_model.requires_grad_(False)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

### Adaptando el modelo

Agregamos una capa lineal al final de la red neuronal. Esta capa sólo tiene 1 neurona de salida, puesto que estamos trabajando con un problema de clasificación binaria.

In [12]:
model = nn.Sequential(
    mobilenet_model,
    nn.Linear(1000, 1)
)

### Preparando función de pérdida y optimizador

Debido a que estamos trabajando con un problema de clasificación binaria, utilizaremos la función de pérdida BCEWithLogitsLoss. El optimizador a utilizar es Adam.

In [ ]:
loss_function = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters())
model = model.to(device)

## Entrenamiento

### Definiendo funciones de entrenamiento

In [ ]:
def get_batch_accuracy(output, y, N):
    zero_tensor = torch.tensor([0]).to(device)
    pred = torch.gt(output, zero_tensor)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

In [ ]:
def train() -> tuple[float, float]:
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)

    return loss, accuracy

In [ ]:
def validate() -> tuple[float, float]:
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)

    return loss, accuracy

### Ejecución del entrenamiento

In [ ]:
epochs = 10
prev_loss = float('inf')
min_delta = 0.001

for epoch in range(epochs):
    train_loss, train_acc = train()
    valid_loss, valid_acc = validate()
    print(f'Epoch {epoch}: Train Loss={train_loss:.4f}, Acc={train_acc:.4f} | Valid Loss={valid_loss:.4f}, Acc={valid_acc:.4f}')

    if abs(prev_loss - valid_loss) < min_delta:
        print('Stopping early')

## Evaluación


In [ ]:
y_test = []
y_pred = []

for x, y in test_loader:
    y_test += [int(y[i]) for i in range(len(y))]

    output = model(x)
    y_pred += [int(output[i]) for i in range(len(y))]

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Barrada", "Barrada"], yticklabels=["No Barrada", "Barrada"])
plt.title("Matriz de Confusión - XGBoost")
plt.xlabel("Predicción")
plt.ylabel("Valor Real")
plt.show()

In [ ]:
print("Reporte de Clasificación - XGBoost")
print(classification_report(y_test, y_pred, target_names=["No Barrada", "Barrada"]))